# Jupyter Notebook to prepare files for analysis

This script prepare files for analysis which will be further used in other jupyter notebooks to evaluate aggregation plots, clusterings of the fibrils, chain labelling, and aggregation matrixes for every frame

Inputs:
Peptides concentration
Number of replica
timestep

In [1]:
concentration = 13
replica_number = 1
dt = 1000
cdt = 100000
temperature = 304
nchains = 2000

# Valuta se fargli scrivere queste info su un file in modo da farli leggere in automatico agli altri script

### Trajectories preparation
#### Here the trajectories are appended into a single one.

In [2]:
folder = f'make_fibril_{nchains}_{concentration}mM_{temperature}K_analysis_test'
#folder = f'make_fibril_{nchains}_{concentration}mM_{temperature}K_{replica_number}'
%cd $folder
%mkdir analysis
%ls

/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_304K_analysis_test
2000_pep-13mM.gro   min_2000-13mM.log              prod_2000-13mM.part0002.log
analysis/           min_2000-13mM.tpr              prod_2000-13mM.part0002.xtc
cg_2000-13mM.edr    min_2000-13mM.trr              prod_2000-13mM.part0003.edr
cg_2000-13mM.gro    posre.itp                      prod_2000-13mM.part0003.log
cg_2000-13mM.log    processed.top                  prod_2000-13mM.part0003.xtc
cg_2000-13mM.tpr    prod_2000-13mM-posre.cpt       prod_2000-13mM.part0004.edr
cg_2000-13mM.trr    prod_2000-13mM-posre.edr       prod_2000-13mM.part0004.log
daint_submit        prod_2000-13mM-posre.gro       prod_2000-13mM.part0004.xtc
ff_aa-posre.mdp     prod_2000-13mM-posre.log       prod_2000-13mM.tpr
ff_aa.mdp           prod_2000-13mM-posre.tpr       prod_2000-13mM_prev.cpt
ff_cg.mdp           prod_2000-13mM-posre.xtc       script_dep.sh
f

In [3]:
import os, os.path
file_list = os.listdir('.')
file_list = list(filter(lambda k: 'part' in k, file_list))
file_list = list(filter(lambda k: 'xtc' in k, file_list))
file_list.sort()
file_list = ' '.join(file_list)
file_list

'prod_2000-13mM.part0001.xtc prod_2000-13mM.part0002.xtc prod_2000-13mM.part0003.xtc prod_2000-13mM.part0004.xtc'

In [4]:
trjcat = f'gmx_mpi trjcat -f {file_list} -o prod_{nchains}-{concentration}mM_full'
!{trjcat}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjcat, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Two trajectories are made with timestep and one includes also the pbc. The one without pbc will be used for cluststize.

NOPBC trajectory

In [5]:
string = f'echo -e "0\n0\n" | gmx_mpi trjconv -s prod_{nchains}-{concentration}mM.tpr -f prod_{nchains}-{concentration}mM_full.xtc -o NOPBC_prod_{nchains}-{concentration}mM-dt{dt} -dt {dt} -pbc mol -center -nice 0'
!{string}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjconv, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                      

Clustsize trajectory

In [6]:
string = f'echo -e "0\n0\n" | gmx_mpi trjconv -s prod_{nchains}-{concentration}mM.tpr -f prod_{nchains}-{concentration}mM_full.xtc -o analysis/prod_{nchains}-{concentration}mM-dt{cdt} -dt {cdt} -nice 0'
!{string}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjconv, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                      

### Clustsize part
#### Writing the maxclust.ndx of the last frame for chain labelling

In [18]:
%cd analysis/
string = f'ln -s ../{nchains}_pep-{concentration}mM.gro'
!{string}
string = f'ln -s ../prod_{nchains}-{concentration}mM.tpr'
!{string}
string = f'ln -s ../NOPBC_prod_{nchains}-{concentration}mM-dt{dt}.xtc'
!{string}
%cd ..

[Errno 2] No such file or directory: 'analysis/'
/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_304K_analysis_test/analysis
ln: failed to create symbolic link './2000_pep-13mM.gro': File exists
ln: failed to create symbolic link './prod_2000-13mM.tpr': File exists


In [23]:
%cd analysis/
# Saving the last frame coordinates for maxclust.ndx
import mdtraj as md
trajectory = md.load(f'NOPBC_prod_{nchains}-{concentration}mM-dt{dt}.xtc', top = f'{nchains}_pep-{concentration}mM.gro')

print(trajectory[-1])
trajectory[-1].save('last_frame.xtc')
trajectory = 0 # I hope this will save some RAM
%cd ..

<mdtraj.Trajectory with 1 frames, 170000 atoms, 22000 residues, and unitcells>


In [20]:
%cd analysis/
string = f'gmx_mpi clustsize -f last_frame.xtc -s prod_{nchains}-{concentration}mM.tpr -mol -nice 0 -cut 0.4 -mcn last_frame.ndx'
!{string}
!{'rm *clust* histo-time.dat csize* temp.xvg'}
%cd ..

Invalid MIT-MAGIC-COOKIE-1 key                :-) GROMACS - gmx clustsize, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                     

In [24]:
import time
start_time = time.time()
%cd analysis/
string = f'gmx_mpi clustsize -f prod_{nchains}-{concentration}mM-dt{cdt}.xtc -dt {cdt} -s prod_{nchains}-{concentration}mM.tpr -mol -hct matrix-dt{cdt}.dat -nice 0 -cut 0.4'
!{string}

print(time.time() - start_time)
%cd ..

1617812082.4155436
Invalid MIT-MAGIC-COOKIE-1 key                :-) GROMACS - gmx clustsize, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
  